In [0]:
!pip install flask_ngrok
!pip install tensorflow==1.15
!pip install "tensorflow_hub>=0.6.0"
!pip3 install tensorflow_text==1.15
!pip install -U flask_cors

Requirement already up-to-date: flask_cors in /usr/local/lib/python3.6/dist-packages (3.0.8)


In [0]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.porter import *
import re
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#Module 1 - Fetch Reviews

In [0]:
import requests
import json
def getReviews(place):
  result={}
  
  # place = input("Enter place name: ")
  placeId = requests.get('https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input='+place+'&inputtype=textquery&fields=place_id,photos,formatted_address,name,rating,opening_hours,geometry&key=YOUR_API_KEY')
  idJsonText = placeId.text
  idText = json.loads(idJsonText)
  place_id = idText['candidates'][0]['place_id']
  placeDetails = requests.get('https://maps.googleapis.com/maps/api/place/details/json?place_id='+place_id+'&fields=name,rating,reviews&key=YOUR_API_KEY')
  jsonText = placeDetails.text
  text = json.loads(jsonText)
  revList = text['result']['reviews']
  passage = ""
  for rev in revList:
    review = rev['text']
    passage = passage + review

  return passage


#Module 2 - Text Preprocessing

In [0]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [0]:
contractions_re = re.compile('(%s)' % '|'.join(contractions.keys()))

def expand_contractions(s, contractions_dict=contractions):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)
 


In [0]:
def create_freq_table(text_string):
    stopwords_list = set(stopwords.words('english'))
    
    words = word_tokenize(text_string)
    
    ps = PorterStemmer()
    
    freq_table = {}
    
    for word in words:
        #stem word 
        word = ps.stem(word)
        
        #remove stopwords
        if word in stopwords_list: 
            continue
        elif word in freq_table:
            freq_table[word] += 1
        else:
            freq_table[word] = 1
            
    return freq_table



In [0]:
def score_sentences(sentences, freq_table):
    
    sentence_value = {}
    
    for sentence in sentences:
        word_count_in_sentence = len(word_tokenize(sentence))
        
        for wordValue in freq_table:
            
            if wordValue.lower() in sentence.lower():                
                if sentence in sentence_value:
                    sentence_value[sentence] += freq_table[wordValue]
                else:
                    sentence_value[sentence] = freq_table[wordValue]

        sentence_value[sentence] = sentence_value[sentence] // word_count_in_sentence
    return sentence_value

def find_average_score(sentence_value):
    sum_values = 0
    
    for entry in sentence_value:
        sum_values += sentence_value[entry]
        
    average = int(sum_values/len(sentence_value))  
    return average

In [0]:
def generate_summary(sentences, sentence_value, threshold):
    sentence_count = 0   
    summary = ''
    
    for sentence in sentences:
        if sentence in sentence_value and sentence_value[sentence] > threshold:
            summary += " " + sentence
            sentence_count += 1
            
    return summary 
            
            

#Module 3 - Summarizer 1

In [0]:

def summary_1(passage):
  sentences = sent_tokenize(passage)    
  sentences = [expand_contractions(i) for i in sentences]
  sentences = [re.sub('\n', '', i) for i in sentences]
  freq_table = create_freq_table(" ".join(sentences))

  sentence_scores = score_sentences(sentences, freq_table)

  threshold = find_average_score(sentence_scores)

  summary = generate_summary(sentences, sentence_scores, 1.0 * threshold)

  #result["summary1"] = re.sub('\n','',summary)
  return summary
 

#Module 4 - Summarizer 2



In [0]:
def summary_2(passage):

  sentences = sent_tokenize(passage)    
  sentences = [expand_contractions(i) for i in sentences]
  sentences = [re.sub('\n', '', i) for i in sentences]

  module_url = "https://tfhub.dev/google/universal-sentence-encoder/2"

  embed = hub.Module(module_url)

  # Reduce logging output.
  tf.logging.set_verbosity(tf.logging.ERROR)

  with tf.Session() as session:
      session.run([tf.global_variables_initializer(), tf.tables_initializer()])
      message_embeddings = session.run(embed(sentences))
  

  #generate cosine similarity matrix
  sim_matrix = cosine_similarity(message_embeddings)

  #create graph and generate scores from pagerank algorithms
  nx_graph = nx.from_numpy_array(sim_matrix)
  scores = nx.pagerank(nx_graph)

  ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    
  num_of_sentences = 5
      
  summary = " ".join([i[1] for i in ranked_sentences[:num_of_sentences]])
  #result["summary2"]=summary2

  return summary


In [0]:

from flask import Flask
from flask import request
from flask_cors import CORS, cross_origin
from flask_ngrok import run_with_ngrok


app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run
CORS(app)

@app.route('/getInput/<place>',methods = ['GET', 'POST'])
@cross_origin()
def get_placename(place):
    reviews={}
    passage = getReviews(place)
    reviews["passage"]= getReviews(place)
    reviews["summary1"] = summary_1(passage)
    reviews["summary2"] = summary_2(passage)
    
    return reviews

# for / root, return Hello Word
@app.route("/")

def root():
  
    return f"Hello World! "

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d371a04c.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [18/May/2020 14:32:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2020 14:32:12] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/May/2020 14:32:45] "GET /getInput/lulu%20mall HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/May/2020 14:32:45] "GET /getInput/lulu%20mall HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2020 14:35:47] "GET /getInput/nit%20calicut HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/May/2020 14:35:47] "GET /getInput/nit%20calicut HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2020 14:35:53] "GET /getInput/nit%20calicut HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/May/2020 14:35:53] "GET /getInput/nit%20calicut HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2020 14:35:59] "GET /getInput/calicut%20paragon HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/May/2020 14:35:59] "GET /getInput/calicut%20paragon HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2020 14:36:12] "GET /getInput/calicut%20paragon HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/May/2020 14:36:12] "GET /getInput/c